In [3]:
import pymongo
import schedule
import time
from datetime import datetime
import redis
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import sys

# Establish connection to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Establish connection to Redis using Docker container host
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)

# Access the database
db = client['yelp']

# Access the collections
review_collection = db['review']
business_collection = db['business']
admin_collection = db['admin']

In [4]:
# SEND EMAIL NOTIFICATION TO ADMIN IF THRESHOLD IS EXCEEDED

# Get the business name as input
username_input = input("Enter admin username: ")
password_input = input("Enter admin password: ")
print("Checking for new low-rated reviews...")

def send_email(receiver_email, subject, body):
    sender_email = "eunicelimuria@gmail.com"  # Replace with your email
    sender_password = "qtyo xhpj mpuk znjt"   # Replace with your email password

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject

    message.attach(MIMEText(body, "plain"))

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, message.as_string())


previous_review_ids = set()  # Storing previous review IDs

def notify_low_rating_reviews(username, password):
    existing_admin = admin_collection.find_one({"admin_username": username})
    if existing_admin is not None:
        pass
    else:
        raise ValueError("Wrong username. Please enter the right username and password.")

    threshold_percentage = admin_collection.find_one({"admin_username": username})['threshold_percentage']
    last_n_reviews = admin_collection.find_one({"admin_username": username})['last_n_reviews']
    business_id = admin_collection.find_one({"admin_username": username})['business_id']
    business_name = business_collection.find_one({"business_id": business_id})['name']

    if admin_collection.find_one({"admin_username": username})['admin_password'] != password:
        raise ValueError("Wrong password. Please enter the right username and password.")

    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


    admin = admin_collection.find_one({"business_id": business_id})
    if not admin:
        return "Admin not found"
    admin_email = admin['admin_email']

    # Find the last 'n' reviews for the given business_id based on the "date" field
    recent_reviews = list(review_collection.find(
        {"business_id": business_id}
    ).sort([("date", -1)]).limit(last_n_reviews))

    print("Recent reviews:")
    for review in recent_reviews:
        print(f"Review: {review['_id']} - Stars: {review['stars']} - Date: {review['date']}")

    # Get the set of current review IDs
    current_review_ids = {review['_id'] for review in recent_reviews}

    # Check if there are new reviews by comparing review IDs
    new_reviews = current_review_ids - previous_review_ids

    # Update previous_review_ids with the current review IDs
    previous_review_ids.update(current_review_ids)

    if new_reviews:  # If there are new reviews, proceed to send an email
        # Calculate the percentage of reviews with stars 1 or 2 out of the last 'n' reviews
        low_rating_count = sum(1 for review in recent_reviews if review['stars'] in [1, 2])
        total_reviews = len(recent_reviews)
        percentage = (low_rating_count / total_reviews) * 100 if total_reviews > 0 else 0

        # Notify if the threshold percentage is exceeded
        if percentage >= threshold_percentage:
            redis_channel = "admin_negative_reviews_channel"
            redis_message = f"Business: {business_name} received {percentage} negative reviews"
            redis_client.publish(redis_channel, redis_message)

            # Send email notification
            email_subject = "Negative Reviews Notification"
            email_body = f"Dear {business_name} Admin,\n\nYour business has received {percentage:.2f}% negative reviews of the last {last_n_reviews} reviews.\n\nChecked on {current_date}\n\nSincerely,\nYelp Review System."
            send_email(admin_email, email_subject, email_body)

            return f"Checked on {current_date}. Threshold exceeded: {percentage:.2f}% of the last {last_n_reviews} reviews for '{business_name}' have low ratings."

    return None


def job():
    try:
        result = notify_low_rating_reviews(username_input, password_input)
        if result:
            print(result)
    except ValueError as err:
        print(err)
        sys.exit()

# Schedule the job to run every second (adjust as needed)
schedule.every(1).second.do(job)

# Run the job initially
job()

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(30)

Checking for new low-rated reviews...
Recent reviews:
Review: 821d05f2-ba2c-4c84-b36a-1e3bb47cff90 - Stars: 4 - Date: 2023-12-05 10:39:22
Review: 656cc6db5d1141c25dd0a54a - Stars: 1 - Date: 2021-03-06 07:18:00
Review: 656cc6d75d1141c25dcfca79 - Stars: 2 - Date: 2020-06-27 23:31:58
Review: 656cc6d35d1141c25dced8f4 - Stars: 1 - Date: 2020-06-26 19:22:36
Checked on 2023-12-05 11:18:34. Threshold exceeded: 75.00% of the last 4 reviews for 'Sonic Drive-In' have low ratings.
Recent reviews:
Review: 821d05f2-ba2c-4c84-b36a-1e3bb47cff90 - Stars: 4 - Date: 2023-12-05 10:39:22
Review: 656cc6db5d1141c25dd0a54a - Stars: 1 - Date: 2021-03-06 07:18:00
Review: 656cc6d75d1141c25dcfca79 - Stars: 2 - Date: 2020-06-27 23:31:58
Review: 656cc6d35d1141c25dced8f4 - Stars: 1 - Date: 2020-06-26 19:22:36
Recent reviews:
Review: 821d05f2-ba2c-4c84-b36a-1e3bb47cff90 - Stars: 4 - Date: 2023-12-05 10:39:22
Review: 656cc6db5d1141c25dd0a54a - Stars: 1 - Date: 2021-03-06 07:18:00
Review: 656cc6d75d1141c25dcfca79 - Sta

In [ ]:
# Storing previous review IDs
previous_review_ids = set()

def email_exists(email):
    existing_admin = admin_collection.find_one({"admin_email": email})
    return existing_admin is not None

def business_exists(business_name):
    existing_business = admin_collection.find_one({"business_name": business_name})
    return existing_business is not None

def send_email(receiver_email, subject, body):
    sender_email = "eunicelimuria@gmail.com"  # Replace with your email
    sender_password = "qtyo xhpj mpuk znjt"   # Replace with your email password

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject

    message.attach(MIMEText(body, "plain"))

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, message.as_string())

def notify_low_rating_reviews(username, password, threshold_percentage, last_n_reviews, business_id, admin_email):
    if admin_collection.find_one({"admin_username": username})['admin_password'] != password:
        raise ValueError("Wrong password. Please enter the right username and password.")
    
    threshold_percentage = admin_collection.find_one({"admin_username": username})['threshold_percentage']
    last_n_reviews = admin_collection.find_one({"admin_username": username})['last_n_reviews']
    business_id = admin_collection.find_one({"admin_username": username})['business_id']
    business_name = business_collection.find_one({"business_id": business_id})['name']

    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    admin = admin_collection.find_one({"business_id": business_id})
    if not admin:
        return "Admin not found"
    
    # Find the last 'n' reviews for the given business_id based on the "date" field
    recent_reviews = list(review_collection.find(
        {"business_id": business_id}
    ).sort([("date", -1)]).limit(last_n_reviews))

    print(f"Recent reviews for Admin {username}:")
    for review in recent_reviews:
        print(f"Review: {review['_id']} - Stars: {review['stars']} - Date: {review['date']}")

    # Get the set of current review IDs
    current_review_ids = {review['_id'] for review in recent_reviews}

    # Check if there are new reviews by comparing review IDs
    new_reviews = current_review_ids - previous_review_ids

    # Update previous_review_ids with the current review IDs
    previous_review_ids.update(current_review_ids)

    admin_name = admin['admin_name']

    if new_reviews:  # If there are new reviews, proceed to send an email
        # Calculate the percentage of reviews with stars 1 or 2 out of the last 'n' reviews
        low_rating_count = sum(1 for review in recent_reviews if review['stars'] in [1, 2])
        total_reviews = len(recent_reviews)
        percentage = (low_rating_count / total_reviews) * 100 if total_reviews > 0 else 0

        # Notify if the threshold percentage is exceeded
        if percentage >= threshold_percentage:
            redis_channel = "admin_negative_reviews_channel"
            redis_message = f"Business: {business_name} received {percentage} negative reviews"
            redis_client.publish(redis_channel, redis_message)

            # Send email notification
            email_subject = "Negative Reviews Notification"
            email_body = f"Dear {admin_name},\n\nYour business '{business_name}' has received {percentage:.2f}% negative reviews of the last {last_n_reviews} reviews.\n\nChecked on {current_date}\n\nSincerely,\nYelp Review System."
            send_email(admin_email, email_subject, email_body)

            return f"Checked on {current_date}. Threshold exceeded: {percentage:.2f}% of the last {last_n_reviews} reviews for '{business_name}' have low ratings."

    return None

def job():
    try:
        admins = admin_collection.find({})
        for admin in admins:
            username = admin['admin_username']
            password = admin['admin_password']
            threshold_percentage = admin['threshold_percentage']
            last_n_reviews = admin['last_n_reviews']
            business_id = admin['business_id']
            admin_email = admin['admin_email']

            result = notify_low_rating_reviews(username, password, threshold_percentage, last_n_reviews, business_id, admin_email)
            if result:
                print(result)
    except ValueError as err:
        print(err)
        sys.exit()

# Schedule the job to run every second (adjust as needed)
schedule.every(1).second.do(job)

# Run the job initially
job()

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(30)